#도서 객체 수집

In [18]:
class Book:
    def __init__(self, rank, title, author, price):
        self.rank = rank
        self.title = title
        self.author = author
        self.price = price

    def __str__(self):
        return f"{self.rank}, {self.title}, {self.author}, {self.price}"
    
    def to_dict(self):
        return {'rank':self.rank, 
                'title':self.title, 
                'author':self.author, 
                'price':self.price}
    
    def to_list(self):
        return [self.rank, 
                self.title, 
                self.author, 
                self.price]

In [19]:
#yesBestList > li:nth-child(1) > div > div.item_info

In [20]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.yes24.com/product/category/daybestseller?categoryNumber=001&pageNumber=1&pageSize=24&type=day'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
best_list_el = soup.select('#yesBestList div.item_info')
len(best_list_el)



24

In [21]:
best_list_el[0]

<div class="item_info">
<div class="info_row info_keynote">
<span class="gd_keynote" id="spanGdKeynote">
<!-- 클래스24 상품일 경우 -->
</span>
<script type="text/javascript">
        if ($('#spanGdKeynote').children().length == 0) {
            $('#spanGdKeynote').remove();
        }
    </script>
</div>
<div class="info_row info_name">
<span class="gd_res">[만화]</span>
<a class="gd_name" href="/product/goods/150111139" onclick="wiseLogV2('BS', '003_006_001', ''); ">명탐정 코난 107 특장판</a>
<span class="gd_feature">
                        [
<span class="feature">구성 : 극장판 [척안의 잔상] 티저 일러스트 아크릴 스탠드+일러스트 PET 스탠드</span>                        ]
                    </span>
<a class="bgYUI ico_nWin" href="/product/goods/150111139" onclick="wiseLogV2('BS', '003_006_001', ''); " target="_blank">명탐정 코난 107 특장판 새창이동</a>
</div>
<div class="info_row info_pubGrp">
<span class="authPub info_auth" onclick="wiseLogV2('BS', '003_006_002', '');">
<a href="https://www.yes24.com/product/search?domain=ALL&amp;query=%25EC

In [22]:
book_list = []

for i, item in enumerate(best_list_el):
    title = item.select_one('div.info_name > a').text
    author = item.select_one('.info_auth > a').text #span tag는 입력 안해도 됨
    price = item.select_one('div.info_price .txt_num').text #strong class -- strong 여러개일 수 있으니까 '.' 으로 함
    
    book_list.append(Book(i + 1, title, author,price))

for book in book_list:
    print(book)


1, 명탐정 코난 107 특장판, 아오야마 고쇼, 17,100원
2, 혼모노, 성해나, 16,200원
3, 자몽살구클럽, 한로로, 10,800원
4, 명탐정 코난 107 일반판, 아오야마 고쇼, 5,400원
5, 가공범, 히가시노 게이고, 19,800원
6, 조국의 공부, 조국, 16,920원
7, 류수영의 평생 레시피, 류수영, 22,500원
8, 편안함의 습격, 마이클 이스터, 19,800원
9, 2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급), 최태성, 17,550원
10, 안녕이라 그랬어, 김애란, 15,120원
11, ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC, ETS, 17,820원
12, 박곰희 연금 부자 수업, 박곰희, 18,900원
13, 료의 생각 없는 생각, 료, 18,000원
14, ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC, ETS, 17,820원
15, 다크 심리학, 다크 사이드 프로젝트, 19,710원
16, 청춘의 독서, 유시민, 17,010원
17, 모순, 양귀자, 11,700원
18, 해커스 토익 기출 VOCA (보카), David Cho, 11,610원
19, 어른의 행복은 조용하다, 태수, 16,020원
20, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상, 최태성, 14,850원
21, 단 한 줄만 내 마음에 새긴다고 해도, 나민애, 21,420원
22, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하, 최태성, 14,400원
23, 2026 유휘운 행정법총론 요약노트+기출문제 (요.플.), 유휘운, 31,500원
24, 2026 써니 행정법총론 기본서, 박준철, 38,700원


In [23]:
query = '''
CREATE TABLE IF NOT EXISTS BOOKS(
    RANK INTEGER PRIMARY KEY,
    TITLE TEXT,
    AUTHOR TEXT,
    PRICE INTEGER
)
'''

In [24]:
import sqlite3 
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()
cursor.execute(query)
conn.commit()
conn.close()

In [25]:
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

In [26]:
# ✅ 책 수집 코드에서 rank 값이 페이지마다 리셋되지 않도록 아래처럼 유지해야 해
rank = 1  # 전체 순위 1부터 시작
for page in range(1, 6): 
    ...
    for item in best_list_el:
        ...
        book_list.append(Book(rank, title, author, price))
        rank += 1  # 이게 꼭 있어야 함!!!


In [27]:
book_list = []

for i, item in enumerate(best_list_el):
    title = item.select_one('div.info_name > a').text.strip()
    author = item.select_one('.info_auth > a').text.strip()
    price = item.select_one('div.info_price .txt_num').text.strip()

    book_list.append(Book(i + 1, title, author, price))


In [28]:
def fetch_book_list_from_db():
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    try:
        cursor.execute('SELECT * FROM books')    
        rows = cursor.fetchall()
        return rows
    finally:
        conn.close()

In [29]:
books = fetch_book_list_from_db()
for book in books:
    print(book)

(1, '혼모노', '성해나', '16,200원')
(2, '가공범', '히가시노 게이고', '19,800원')
(3, '류수영의 평생 레시피', '류수영', '22,500원')
(4, '편안함의 습격', '마이클 이스터', '19,800원')
(5, '2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급)', '최태성', '17,550원')
(6, '다크 심리학', '다크 사이드 프로젝트', '19,710원')
(7, '자몽살구클럽', '한로로', '10,800원')
(8, '박곰희 연금 부자 수업', '박곰희', '18,900원')
(9, 'ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC', 'ETS', '17,820원')
(10, 'ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC', 'ETS', '17,820원')
(11, '2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상', '최태성', '14,850원')
(12, '2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하', '최태성', '14,400원')
(13, '안녕이라 그랬어', '김애란', '15,120원')
(14, '사기열전 1,2 세트', '사마천', '45,900원')
(15, '푸른 사자 와니니 8', '이현', '12,420원')
(16, '마흔, 더 늦기 전에 생각의 틀을 리셋하라', '박근필', '16,020원')
(17, '모순', '양귀자', '11,700원')
(18, '단 한 줄만 내 마음에 새긴다고 해도', '나민애', '21,420원')
(19, '소년이 온다', '한강', '13,500원')
(20, '해커스 토익 기출 VOCA (보카)', 'David Cho', '11,610원')
(21, '료의 생각 없는 생각', '료', '18,000원')
(22, '청춘의 독서', '유시민', '17,010원')
(23, '첫 여름, 완주', '김금희', '15,300원')


In [30]:
base_url = 'https://www.yes24.com/product/category/daybestseller?categoryNumber=001&pageSize=24&pageNumber='
page_no = 3
book_list = []
for page in range(1, page_no+1):
    # 페이지 연결    
    url = f'{base_url}{page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    best_list_el = soup.select('#yesBestList div.item_info')
# 요소추출
for i, item in enumerate(best_list_el):
    title = item.select_one('div.info_name > a').text
    author = item.select_one('.info_auth > a').text #span tag는 입력 안해도 됨
    price = item.select_one('div.info_price .txt_num').text #strong class -- strong 여러개일 수 있으니까 '.' 으로 함
    
    book_list.append(Book(i + 1, title, author,price))
    
len(book_list)

24

In [31]:


import requests
from bs4 import BeautifulSoup

# book 클래스 정의 : 각 책 정보를 객체로 저장
class Book: 
    def __init__(self, rank, title, author, price):
        self.rank = rank
        self.title = title
        self.author = author
        self.price = price
		
		# to_tuple() 메서드 : DB INSERT용 튜플 변환
    def to_tuple(self):
        return (self.rank, self.title, self.author, self.price)

base_url = 'https://www.yes24.com/product/category/daybestseller?categoryNumber=001&pageSize=24&pageNumber='
book_list = []


rank = 1  # 전체 순위 1부터 시작
for page in range(1, 6):   # 1~5페이지 반복(한 페이지당 24권 × 5 = 최대 120권 확보 가능)
    url = f'{base_url}{page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    best_list_el = soup.select('#yesBestList div.item_info')

    for item in best_list_el:                       #.text.strip(): 불필요한 공백 제거 완벽
        title = item.select_one('div.info_name > a').text.strip()    
        author = item.select_one('.info_auth > a').text.strip()
        price_raw = item.select_one('div.info_price .txt_num').text.strip()
        price = int(price_raw.replace(",", "").replace("원", ""))


        book_list.append(Book(rank, title, author, price))
        rank += 1  # rank 전역 증가 : 전체 순위 1~100 매기기 OK

        if rank > 100:    # break 조건 : 100권 도달하면 중단
            break 
    if rank > 100:
        break  

print(f"총 수집된 책 수: {len(book_list)}")



총 수집된 책 수: 100


In [33]:
import mysql.connector

insert_query = '''
INSERT INTO books (book_rank, title, author, price)
VALUES (%s, %s, %s, %s)
'''

# 1. DB 연결
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Qpswutm45!",
    database="my_db",
    charset="utf8mb4"
)

# 2. 커서 생성
cursor = conn.cursor()

# 3. 테이블 비우기
cursor.execute("DELETE FROM books")

# 4. INSERT 반복
for book in book_list:
    cursor.execute(insert_query, book.to_tuple())

# 5. 저장 + 종료
conn.commit()
conn.close()

print("✅ 전체 삭제 후 재삽입 완료!")


✅ 전체 삭제 후 재삽입 완료!
